In [5]:
import pandas as pd
import datetime
from selenium import webdriver
import time
import sqlite3



def scrape_ah(ah_webpages, df_total):
    for page, category in ah_webpages:
        product = []
        size = []
        price_frac = []
        price_int = []
        discount = []
        print(page)
        driver = webdriver.Chrome(executable_path=r'/usr/local/bin/chromedriver')
        driver.get(page)

        # wait = WebDriverWait(driver, 20)
        time.sleep(5)

        child_tags = driver.find_elements_by_xpath(".//*[@class=' row collapse product__content product__content--link']")

        for i in range(len(child_tags)):
            product.append(child_tags[i].find_element_by_xpath(".//*[@class='product-description__title  -multiline']").text)
            size.append(child_tags[i].find_element_by_xpath(".//*[@class='product-description__unit-size -multiline']").text)
            price_frac.append(child_tags[i].find_element_by_xpath(".//*[@class='price__fractional']").text)
            price_int.append(child_tags[i].find_element_by_xpath(".//*[@class='price__integer']").text)
            try:
                discount.append(child_tags[i].find_element_by_xpath(".//*[@class='discount-block']").text)
            except:
                discount.append(0)

        df = pd.DataFrame({'product': product, 'size': size, 'price_frac': price_frac, 'price_int': price_int, 'discount': discount})
        time.sleep(2)
        driver.close()
        df
        df['price'] = (df['price_int'] + '.' + df['price_frac']).astype('float')
        df = df.drop(['price_frac', 'price_int'], axis=1)
        df['date'] = datetime.datetime.now().strftime("%Y-%m-%d")
        df['category'] = category

        df_total = pd.concat([df_total, df], sort=True)
    return df_total.drop_duplicates()



def to_db(DF_AH):
    sqlite_file = '/Users/wcasey/Documents/personal_projects/projects/ah/ah.db'
    conn = sqlite3.connect(sqlite_file)
    DF_AH.to_sql(name='ah_inventory', con=conn, if_exists='append', index=False)

    #commit changes to the db and close the connection
    conn.commit()
    conn.close()



def main():
    df_total = pd.DataFrame({'product': [], 'size': [], 'price': [], 'date': [],'discount': [] , 'category': []})
    ah_webpages = [
              ('https://www.ah.nl/producten/aardappel-groente-fruit', 'produce'),
              ('https://www.ah.nl/producten/vlees-kip-vis-vega', 'meats'),
              ('https://www.ah.nl/producten/kaas-vleeswaren-delicatessen', 'deli-cheese'),
              ('https://www.ah.nl/producten/zuivel-eieren', 'dairy-eggs'),
              ('https://www.ah.nl/producten/frisdrank-sappen-koffie-thee','drinks-coffee-tea' ),
              ('https://www.ah.nl/producten/drogisterij-baby', 'drugstore-baby')
              ]

    DF_AH = scrape_ah(ah_webpages, df_total)
#     to_db(DF_AH)
    return DF_AH


if __name__ == '__main__':
    df = main()

https://www.ah.nl/producten/aardappel-groente-fruit
https://www.ah.nl/producten/vlees-kip-vis-vega
https://www.ah.nl/producten/kaas-vleeswaren-delicatessen
https://www.ah.nl/producten/zuivel-eieren
https://www.ah.nl/producten/frisdrank-sappen-koffie-thee
https://www.ah.nl/producten/drogisterij-baby


In [6]:
df

,category,date,discount,price,product,size
0,produce,2018-10-20,0,1.79,Ar­la Lact­of­ree crè­me fraî­che,200 g
1,produce,2018-10-20,0,1.99,AH Ge­sne­den cham­pig­nons,250 g
2,produce,2018-10-20,0,1.25,AH Plat­te pe­ter­se­lie,15 g
3,produce,2018-10-20,0,1.09,AH Groen­te­bouil­lon,100 g
4,produce,2018-10-20,0,1.69,AH Fijn­ge­sne­den an­dij­vie voor­deel,500 g
5,produce,2018-10-20,0,1.39,AH Boe­ren­kool,500 g
6,produce,2018-10-20,0,2.00,AH Biet­jes ge­kookt,500 g
7,produce,2018-10-20,0,2.00,AH Bi­o­lo­gisch Ge­rasp­te biet­jes,400 g
8,produce,2018-10-20,0,1.29,AH Cham­pig­nons voor­deel,400 g
10,produce,2018-10-20,0,1.69,AH Kool­raap,400 g
